# Save and load model

當保存和加載模型時，需要熟悉三個核心功能：

1. ``torch.save``：將序列化對象保存到硬碟。此函數使用Python的pickle模組進行序列化。使用此函數可以保存如模型、tensor、字典等各種對象
2. ``torch.load``：使用pickle的unpickling功能將pickle對象文件反序列化到記憶體。此功能還可以有助於設備加載數據
3. ``torch.nn.Module.load_state_dict``：使用反序列化函數 state_dict 來加載模型的參數字典

## state_dict

* ``torch.nn.Module``模型的可學習參數（即權重和偏差）包含在模型的參數中，（使用model.parameters()可以進行訪問）

* ``state_dict``是Python字典對象，它將每一層映射到其參數張量。注意，只有具有可學習參數的層（如卷積層，線性層等）的模型才具有state_dict這一項。目標優化torch.optim也有state_dict屬性，它包含有關優化器的狀態訊息，以及使用的超參數。

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print('\n')
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])


Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


## Save/Load ``state_dict`` (Recommended)

* 保存好模型用來推斷的時候，只需要保存模型學習到的參數，使用torch.save()函數來保存模型state_dict，給模型恢復提供最大的靈活性

* 在 PyTorch 中最常見的模型保存使``.pt``或者是``.pth``

* 在運行推理之前，務必調用model.eval()去設置 dropout 和 batch normalization 層為評估模式。如果不這麼做，可能導致模型推斷結果不一致

In [ ]:
# Save

torch.save(model.state_dict(), PATH)

In [ ]:
# Load

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

## Save/Load Entire Model

* 此部分保存/加載過程使用最直觀的語法並涉及最少量的程式碼。以 Python ``pickle`` 模組來保存模型。這種方法的缺點是序列化數據受限於某種特殊的類而且需要確切的字典結構。這是因為pickle無法保存模型類本身。相反，它保存包含類的文件的路徑，該文件在加載時使用。因此，當在其他項目使用或者重構之後，你的程式碼可能會以各種方式中斷

In [ ]:
# Save

torch.save(model, PATH)

In [ ]:
# Load

# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

# 其他方式參考官網 [https://pytorch.org/tutorials/beginner/saving_loading_models.html]